# TP Vision Master - Calibration de caméras et incrustation (Partie 2)
Ce notebook est une conversion du PDF contenant des exercices pour la calibration de caméras et l'incrustation d'images. Il inclut les codes et explications du TP.

**Objectifs :**
- Calibrer une caméra
- Passer du repère monde au repère caméra et inversement
- Proposer un scénario d'incrustration d'une image personnelle

**Attention :** Interdiction d'utiliser la fonction utilisée dans la partie 1 :*cv2.projectPoints(My3Dpoints, rvecs[InxdexIm], tvecs[InxdexIm], mtx, dist)*




## Étape 1: Importation des bibliothèques nécessaires

In [ ]:
import numpy as np
import glob
import matplotlib.pyplot as plt
from skimage import io
import cv2
from IPython.display import clear_output
%matplotlib inline


## Étape 2: Chargement des images et test de détection des coins

In [ ]:
FigSize = (10, 10)  # Permet d'ajuster la taille des images affichées

# On extrait un tableau avec l'ensemble des fichiers images d'un dossier
# images = ... (reprendre la méthode du TP1 avec le fichier MireRobot.zip


# Test : lire l'image n°2 de cette séquence
img = cv2.imread(images[1])
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)  # Détection de coins sur image N&B

# Test si coins bien détectés
ret, corners = cv2.findChessboardCorners(gray, (10, 10), None)
if ret:
    cv2.drawChessboardCorners(img, (10, 10), corners, ret)
    fig, ax = plt.subplots(figsize=FigSize)
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    plt.title('Image originale et coins')
    plt.xticks([]), plt.yticks([])
    plt.show()
else:
    print("Impossible de détecter les coins de l'image")


## Étape 3: Préparation des points objet et image

In [ ]:
# Préparer les points objet (ici en cm)
objp = np.zeros((10 * 10, 3), np.float32)
objp[:, :2] = 15 * np.mgrid[0:10, 0:10].T.reshape(-1, 2)  # Dimensions des carreaux : 15x15 mm

# Arrays pour stocker les points objet et image
objpoints = []  # Points 3D dans l'espace réel
imgpoints = []  # Points 2D dans le plan image

# Critère pour optimisation dans les détections de coins
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)


## Étape 4: Calibration de la caméra

In [ ]:
FigSize = (10, 10)  # Taille des images affichées
cpt = 0

for fname in images:
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Détection des coins
    ret, corners = cv2.findChessboardCorners(gray, (10, 10), None)

    # Si coins trouvés, affiner la détection
    if ret:
        cv2.cornerSubPix(gray, corners, (11, 11), (-1, -1), criteria)
        objpoints.append(objp)
        imgpoints.append(corners)
        cv2.drawChessboardCorners(img, (10, 10), corners, ret)
        plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
        plt.title(f'Frame {cpt}')
        plt.show()
        clear_output(wait=True)
    cpt += 1

# Calcul de la matrice intrinsèque et des paramètres extrinsèques
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)

print('Matrice intrinsèque :')
print(mtx)


## Etape 1 : Construire la matrice de projection : Matrices mtx, R et T

Après la calibration, nous obtenons les éléments suivants :

- **mtx** : Matrice intrinsèque de la caméra (3x3), contenant des paramètres tels que la distance focale et le point principal.
- **rvecs** et **tvecs** : Vecteurs rotation et translation associés à chaque image utilisée pour la calibration. Ces vecteurs définissent la relation entre le repère de la caméra et le repère du monde.
- La matrice rotation (**R**) peut être obtenue à partir de `rvecs` avec :
  ```python
  R, _ = cv2.Rodrigues(rvecs[i])
  ```

Pour une image donnée, la transformation d’un point dans le repère monde vers le plan image est définie par :

$$ s.p = \text{mtx} \cdot \left( R | T \right) \cdot P  $$

où :
- $p$ est le point projeté dans le plan image
- $s$ est un facteur d'échelle
- $\text{mtx}$ est la matrice intrinsèque (3x3),
- $(R|T)$est une matrice (3x4) combinant rotation et translation,
- $P$ est un point en coordonnées homogènes dans le repère monde.

## Etape 2 : Projeter un point de la scène
Pour réaliser cette tâche, vous devez utiliser les outils suivants :
- Produit matriciel : `C = np.dot(A, B)`
- Transposée d'une matrice : `A.T`
- Construction d'une matrice (3x4) avec R et T :
  ```python
  Proj = np.zeros((3, 4))
  Proj[0:3, 0:3] = R
  Proj[:, 3] = T.T
  ```
- Pour afficher un point sur une image : `plt.plot(x, y, 'r+')`

### Résumé :
1. Extraire la rotation et translation de l'image courant  
2. Construire la matrice projection \( (R|T) \).
3. Multiplier cette matrice par les coordonnées homogènes \( P \) d’un point du repère monde.
4. Projeter le résultat sur le plan image en divisant par \( z \) (homogénéisation des coordonnées).

## Question 1 : projeter l'origine du repère

In [ ]:
# Compléter ce code pour afficher l'origine du repère de la mire sur l'image courante
i = 0  # Index de l'image
R, _ = cv2.Rodrigues(rvecs[i])
T = tvecs[i]

P = np.array([0, 0, 0, 1])  # Point d'origine dans le repère
Proj = np.zeros((3, 4))
Proj[0:3, 0:3] = ??
Proj[:, 3] = ??

p = np.dot(mtx, np.dot(??, ??))
plt.plot(p[0]/p[2], p[1]/p[2], 'r+')  # Projection du point normalisé


## Question 2 : Incrustation d'une imagette sur le damier

### Objectif
Insérer une imagette de votre choix sur le damier détecté dans l'image.

### Étapes à suivre
1. Lire une imagette (exemple : une image de 100x100 pixels).
2. Définir un tableau 2D \( (2 \times N) \) contenant les coordonnées \( (x, y) \) de chaque pixel de l'imagette.
3. Définir un tableau \( (1 \times N) \) contenant les intensités de chaque pixel de l'imagette.
4. Appliquer une transformation des coordonnées en pixels pour les adapter au repère monde (par exemple, avec un facteur de conversion basé sur la taille des carreaux).
5. Créer un tableau \( P \) représentant les coordonnées \( (X, Y, Z) \) des pixels à incruster.
6. Appliquer la transformation vue à la question précédente pour obtenir les coordonnées projetées.
7. Modifier directement les pixels correspondants dans l'image de la scène, en utilisant les coordonnées projetées (arrondies).

In [ ]:
# Étape 1 : Charger une imagette (par exemple 100x100 pixels)
imagette = cv2.imread('path_to_imagette.png')
if imagette is not None:
    # Redimensionner si nécessaire
    imagette = cv2.resize(imagette, (100, 100))

    # Étape 2 : Générer les coordonnées des pixels de l'imagette
    h, w, _ = imagette.shape
    x, y = np.meshgrid(range(w), range(h))
    pixel_coords = np.vstack((x.ravel(), y.ravel()))  # Tableau 2D (2xN)

    # Étape 3 : Extraire les intensités des pixels
    intensities = imagette.reshape(-1, 3)  # Intensités sous forme (N, 3)

    # Étape 4 : Conversion des coordonnées pour le repère monde
    f = 30 / 100  # Facteur de conversion pour adapter l'échelle (30x30 mm)
    world_coords = np.vstack((f * x.ravel(), f * y.ravel(), np.zeros(w * h)))  # (3xN)

    # Étape 5 : Transformation des points monde vers image
    proj_coords = np.dot(mtx, np.dot(Proj, np.vstack((world_coords, np.ones(world_coords.shape[1])))))
    proj_coords = proj_coords[:2] / proj_coords[2]  # Homogénéisation

    # Étape 6 : Modifier l'image en incrustant les pixels de l'imagette
    for u in range(proj_coords.shape[1]):
        x_proj, y_proj = int(proj_coords[0, u]), int(proj_coords[1, u])
        if 0 <= x_proj < img.shape[1] and 0 <= y_proj < img.shape[0]:
            img[y_proj, x_proj, :] = intensities[u]

    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    plt.title('Image avec imagette incrustée')
    plt.show()
else:
    print('Imagette introuvable. Veuillez fournir un chemin valide.')


## Question 3 :
Sur une l'image de votre choix :


*   Créer une séquence d'images, où l'imagette inscrustée se soulève de la mire au rythme de 5 mm par nouvelle image
*   Modifier la séquence pour ajouter une rotation de 5° de l'imagette pour chaque nouvelle image

